# Find proteins within the 8p loss event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import cnvutils
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans effects
params = cnvutils.load_params(os.path.join("..", "data", "params.json"))
CHROMOSOME = params["CHROMOSOME"]
ARM = params["ARM"]
CIS_OR_TRANS = "trans"

read_path = os.path.join("..", "data", f"{CHROMOSOME}{ARM}_ttest_{CIS_OR_TRANS}_effects.tsv")

ttest_results = pd.\
read_csv(read_path, sep="\t").\
rename(columns={"Name": "protein"})

In [3]:
ttest_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.999725,-0.183906
1,brca,A2M,NP_000005.2,0.999725,0.257953
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.999725,0.343823
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.999725,0.091189
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.999725,0.377982
5,brca,AADAT,NP_057312.1|NP_001273611.1,0.999725,-0.745366
6,brca,AAED1,NP_714542.1,0.999725,-0.333413
7,brca,AAGAB,NP_078942.3|NP_001258815.1,0.999725,0.107454
8,brca,AAK1,NP_055726.3,0.999725,-0.106810
9,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.999725,-0.053913


## Select the proteins with a significant change, and take a detour to make some plots

In [4]:
prots = ttest_results[ttest_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = ttest_results[ttest_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {CIS_OR_TRANS} effects"
).configure_title(
    anchor="middle"
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.FacetChart(...)

## Find how many cancers each protein was different in

In [5]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [6]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,ATP6V1D,NP_057078.1,0.046039,-0.456358,-1.0
1,brca,CNOT8,NP_001288002.1|NP_001288003.1|NP_001288006.1|N...,0.000008,1.224195,1.0
2,brca,KCTD15,NP_001123466.1|NP_076981.2,0.021991,0.824774,1.0
3,colon,AGO2,NaN,0.036195,0.176455,1.0
4,colon,ALDH1B1,NaN,0.027869,0.556227,1.0
5,colon,ARFGAP3,NaN,0.040439,-0.269368,-1.0
6,colon,ATP1B3,NaN,0.018340,0.320327,1.0
7,colon,ATP6V1A,NaN,0.013416,-0.216519,-1.0
8,colon,ATP6V1E1,NaN,0.004252,-0.256006,-1.0
9,colon,ATP6V1F,NaN,0.027869,-0.242232,-1.0


In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
CNOT8,"[brca, lscc, luad]",1.0,3
ATP6V1E1,"[colon, lscc, luad]",-1.0,3
ATP6V1H,"[colon, lscc, luad]",-1.0,3
KCTD15,"[brca, lscc]",1.0,2
AGO2,"[colon, lscc]",1.0,2
ARFGAP3,"[colon, lscc]",-1.0,2
ATP6V1G1,"[colon, lscc]",-1.0,2
CRELD2,"[colon, lscc]",-1.0,2
DDX28,"[colon, lscc]",1.0,2


In [8]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3       3
2      33
1    1170
Name: num_cancers, dtype: int64

## Save results

In [9]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = os.path.join("..", "data", f"common_summary_{CHROMOSOME}{ARM}_{CIS_OR_TRANS}.tsv")
prots_summary.to_csv(output_file, sep="\t")